In [1]:
import wibblywobbly as ww

catalog = ["Mouse", "Cat", "Dog", "Human"]
data = ["mice",  "CAT ", "doggo", "PERSON", 999]

In [2]:
ww.map_list_to_catalog(data, catalog)

,Data,Option1,Score1
0,CAT,Cat,100
1,doggo,Dog,90
2,mice,mice,0
3,PERSON,PERSON,0
4,999,999,0


In [17]:
ww.map_list_to_catalog(data, catalog, thr_accept=95,
                       thr_reject=40, reject_value='Other')

,Data,Option1,Score1,Option2,Score2,Option3,Score3
0,CAT,Cat,100,None,NaN,None,NaN
1,doggo,Dog,90,Mouse,20.0,Cat,0.0
2,mice,Mouse,44,Cat,29.0,Human,22.0
3,PERSON,Other,0,None,NaN,None,NaN
4,999,999,0,None,NaN,None,NaN


In [13]:
ww.map_list_to_catalog(data, catalog, output_format="dictionary")

{'PERSON': 'PERSON', 999: 999, 'mice': 'mice', 'doggo': 'Dog', 'CAT ': 'Cat'}

In [11]:
ww.map_list_to_catalog(data, catalog, output_format="dictionary", 
                       thr_accept=95, thr_reject=40, reject_value='Other',
                       warnings=True)

WOBBLY: mice
	Options: Mouse (44), Cat (29), Human (22)
WOBBLY: doggo
	Options: Dog (90), Mouse (20), Cat (0)


{'PERSON': 'Other', 999: 999, 'mice': 'Mouse', 'doggo': 'Dog', 'CAT ': 'Cat'}